In [ ]:
# 文件很大，不要运行，仅供参考
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import xml.etree.cElementTree as ET
from collections import defaultdict
import re

osm_file = open("chicago_illinois.osm", "r")

street_type_re = re.compile(r'\S+\.?$', re.IGNORECASE)
street_types = defaultdict(int)

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()

        street_types[street_type] += 1

def print_sorted_dict(d):
    keys = d.keys()
    keys = sorted(keys, key=lambda s: s.lower())
    for k in keys:
        v = d[k]
        print "%s: %d" % (k, v) 

def is_street_name(elem):
    return (elem.tag == "tag") and (elem.attrib['k'] == "addr:street")

def audit():
    for event, elem in ET.iterparse(osm_file):
        if is_street_name(elem):
            audit_street_type(street_types, elem.attrib['v'])    
    print_sorted_dict(street_types)    

if __name__ == '__main__':
    audit()

##  修正有效性

In [37]:
"""
Your task is to check the "productionStartYear" of the DBPedia autos datafile for valid values.
The following things should be done:
- check if the field "productionStartYear" contains a year
- check if the year is in range 1886-2014
- convert the value of the field to be just a year (not full datetime)
- the rest of the fields and values should stay the same
- if the value of the field is a valid year in the range as described above,
  write that line to the output_good file
- if the value of the field is not a valid year as described above, 
  write that line to the output_bad file
- discard rows (neither write to good nor bad) if the URI is not from dbpedia.org
- you should use the provided way of reading and writing data (DictReader and DictWriter)
  They will take care of dealing with the header.
  你的任务是检查 DBPedia 自动数据文件的“productionStartYear”并获取有效的值。应该完成以下任务：
-    检查字段“productionStartYear”是否包含年份
-    检查该年份是否在 1886 至 2014 范围内
-    将字段值转换为年份（而不是整个日期时间）
-    字段的其他部分和值应该保持不变
-    如果字段的值是如上所述范围内的有效年份，则将该行写入 output_good 文件中
-    如果字段的值不是如上所述的有效年份，则将该行写入 output_bad 文件中
-    你应该采用提供的数据读取和写入方式（DictReader 和 DictWriter），它们将会对标题进行处理。

You can write helper functions for checking the data and writing the files, but we will call only the 
'process_file' with 3 arguments (inputfile, output_good, output_bad).
你可以编写辅助函数来检查数据并编写文件，但是我们将仅调用包含三个参数（inputfile、output_good、output_bad）的“process_file”文件。
"""
import csv
import pprint

INPUT_FILE = 'autos.csv'
OUTPUT_GOOD = 'autos-valid.csv'
OUTPUT_BAD = 'FIXME-autos.csv'

def process_file(input_file, output_good, output_bad):
    # store data into lists for output
    data_good = []
    data_bad = []
    with open(input_file, "r") as f:
        reader = csv.DictReader(f)
        header = reader.fieldnames
        for row in reader:
            # validate URI value
            if row['URI'].find("dbpedia.org") < 0:
                continue # 跳出本次循环

            ps_year = row['productionStartYear'][:4]
            try: # use try/except to filter valid items
                ps_year = int(ps_year)
                row['productionStartYear'] = ps_year
                if (ps_year >= 1886) and (ps_year <= 2014):
                    data_good.append(row)
                else:
                    data_bad.append(row)
            except ValueError: # non-numeric strings caught by exception
                if ps_year == 'NULL':
                    data_bad.append(row)

    # Write processed data to output files
    with open(output_good, "w") as good:
        writer = csv.DictWriter(good, delimiter=",", fieldnames= header)
        writer.writeheader()
        for row in data_good:
            writer.writerow(row)

    with open(output_bad, "w") as bad:
        writer = csv.DictWriter(bad, delimiter=",", fieldnames= header)
        writer.writeheader()
        for row in data_bad:
            writer.writerow(row) 


def test():

    process_file(INPUT_FILE, OUTPUT_GOOD, OUTPUT_BAD)


if __name__ == "__main__":
    test()

# 习题集
## 审查数据质量

In [118]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
In this problem set you work with cities infobox data, audit it, come up with a
cleaning idea and then clean it up. In the first exercise we want you to audit
the datatypes that can be found in some particular fields in the dataset.
The possible types of values can be:
- NoneType if the value is a string "NULL" or an empty string ""
- list, if the value starts with "{"
- int, if the value can be cast to int
- float, if the value can be cast to float, but CANNOT be cast to int.
   For example, '3.23e+07' should be considered a float because it can be cast
   as float but int('3.23e+07') will throw a ValueError
- 'str', for all other values
在此习题集中，你将处理城市 infobox 数据，对数据进行审核，然后想出清理方法并清理数据。
在第一道练习中，请审核数据集中某些特定字段中的数据类型。
值类型可以是：
-    NoneType，如果值是字符串“NULL”或空字符串“”
-    列表，如果值以“{”开头
-    整型，如果值可以转型为整型
-    浮点型，如果值可以转型为浮点型，但是无法转型为整型。
例如，“3.23e+07”应该被当做浮点型，因为可以转型为浮点型，但是int('3.23e+07') 
将抛出 ValueError
-    “str”，表示其他所有值

The audit_file function should return a dictionary containing fieldnames and a 
SET of the types that can be found in the field. e.g.
{"field1": set([type(float()), type(int()), type(str())]),
 "field2": set([type(str())]),
  ....
}
The type() function returns a type object describing the argument given to the 
function. You can also use examples of objects to create type objects, e.g.
type(1.1) for a float: see the test function below for examples.

audit_file 函数应该返回一个字典，其中包含字段名称和可以在该字段中找到的类型集。例如

{"field1": set([type(float()), type(int()), type(str())]),
 "field2": set([type(str())]),
  ....
}
type() 函数返回的是类型对象，描述了提供给该函数的参数。你还可以使用对象示例创建类型对象，
例如 type(1.1) 表示浮点型：具体示例请参阅下面的测试函数。

Note that the first three rows (after the header row) in the cities.csv file
are not actual data points. The contents of these rows should note be included
when processing data types. Be sure to include functionality in your code to
skip over or detect these rows.
注意，cities.csv 文件的前三行（标题行之后）不是实际的数据点。在处理数据类型时，不应该
包含这些行的内容。确保在代码中包含相关功能，以便跳过或检测出这些行。
"""
import codecs
import csv
import json
import pprint

CITIES = 'cities.csv'

FIELDS = ["name", "timeZone_label", "utcOffset", "homepage", "governmentType_label",
          "isPartOf_label", "areaCode", "populationTotal", "elevation",
          "maximumElevation", "minimumElevation", "populationDensity",
          "wgs84_pos#lat", "wgs84_pos#long", "areaLand", "areaMetro", "areaUrban"]

def audit_file(filename, fields):
    fieldtypes = {}

    # YOUR CODE HERE
    for field in fields:
        fieldtypes_tmp = []
        with open(filename) as f:
            reader = csv.DictReader(f)
            for row in reader:
                # validate URI value
                if row['URI'].find("dbpedia.org") < 0:
                    continue
                try:
                    if row[field][0] == '{':
                        fieldtypes_tmp.append(type([]))
                    elif row[field] == 'NULL' or '':
                        fieldtypes_tmp.append(type(None))
                    elif int(row[field]):
                        fieldtypes_tmp.append(type(int()))
                    else:
                        fieldtypes_tmp.append(type(str()))

                except ValueError:
                    try:
                        if float(row[field]):
                            fieldtypes_tmp.append(type(float()))
                        else:
                            pass
                    except ValueError:
                        fieldtypes_tmp.append(type(str()))
        fieldtypes[field] = set(fieldtypes_tmp)

    return fieldtypes


def test():
    fieldtypes = audit_file(CITIES, FIELDS)

    pprint.pprint(fieldtypes)

    assert fieldtypes["areaLand"] == set([type(1.1), type([]), type(None)])
    assert fieldtypes['areaMetro'] == set([type(1.1), type(None)])
    
if __name__ == "__main__":
    test()


{'areaCode': set([<type 'int'>, <type 'NoneType'>, <type 'str'>]),
 'areaLand': set([<type 'float'>, <type 'list'>, <type 'NoneType'>]),
 'areaMetro': set([<type 'float'>, <type 'NoneType'>]),
 'areaUrban': set([<type 'float'>, <type 'NoneType'>]),
 'elevation': set([<type 'float'>, <type 'list'>, <type 'NoneType'>]),
 'governmentType_label': set([<type 'NoneType'>, <type 'str'>]),
 'homepage': set([<type 'NoneType'>, <type 'str'>]),
 'isPartOf_label': set([<type 'list'>, <type 'NoneType'>, <type 'str'>]),
 'maximumElevation': set([<type 'NoneType'>]),
 'minimumElevation': set([<type 'NoneType'>]),
 'name': set([<type 'list'>, <type 'NoneType'>, <type 'str'>]),
 'populationDensity': set([<type 'float'>, <type 'list'>, <type 'NoneType'>]),
 'populationTotal': set([<type 'int'>, <type 'NoneType'>]),
 'timeZone_label': set([<type 'NoneType'>, <type 'str'>]),
 'utcOffset': set([<type 'int'>,
                   <type 'list'>,
                   <type 'NoneType'>,
                   <type 's

## 修复区域

In [183]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
In this problem set you work with cities infobox data, audit it, come up with a
cleaning idea and then clean it up.
在此习题集中，你将处理城市 infobox 数据，对数据进行审核，然后想出清理方法并清理数据。

Since in the previous quiz you made a decision on which value to keep for the
"areaLand" field, you now know what has to be done.
因为在上一道测验中，你已经决定针对“areaLand”字段保留哪个值，你现在知道该如何操作了。

Finish the function fix_area(). It will receive a string as an input, and it
has to return a float representing the value of the area or None.
You have to change the function fix_area. You can use extra functions if you
like, but changes to process_file will not be taken into account.
The rest of the code is just an example on how this function can be used.
完成函数 fix_area()。它将获得字符串输入，并需要返回表示面积值的浮点值或 None。
你需要更改fix_area 函数。你可以使用其他函数，但是对 process_file 的更改不会计入评估范围。
代码的其余部分只是用来展示可以如何使用该函数的示例。
"""
import codecs
import csv
import json
import pprint
import re
import numpy as np

CITIES = 'cities.csv'

# 选择小数位数更精确的数据
def fix_area(area):
    if area == 'NULL':
        area = None
    elif area[0] == '{':
        areaLen = lambda x: len(re.findall(re.compile('(\d\.\d+?)e'), x)[0]) # 计算小数位数
        area_list = area[1:-1].split('|')
        area_fine = area_list[np.argmax(map(areaLen, area_list))]
        area = float(area_fine)
    else:
        area = float(area)
    return area

def process_file(filename):
    # CHANGES TO THIS FUNCTION WILL BE IGNORED WHEN YOU SUBMIT THE EXERCISE
    data = []

    with open(filename, "r") as f:
        reader = csv.DictReader(f)

        #skipping the extra metadata
        for i in range(3):
            l = reader.next()

        # processing file
        for line in reader:
            # calling your function to fix the area value
            if "areaLand" in line:
                line["areaLand"] = fix_area(line["areaLand"])
            data.append(line)

    return data


def test():
    data = process_file(CITIES)

    print "Printing three example results:"
    for n in range(5,8):
        pprint.pprint(data[n]["areaLand"])

    assert data[3]["areaLand"] == None        
    assert data[8]["areaLand"] == 55166700.0
    assert data[20]["areaLand"] == 14581600.0
    assert data[33]["areaLand"] == 20564500.0    


if __name__ == "__main__":
    test()

Printing three example results:
None
101787000.0
31597900.0


## 修复姓名

In [196]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
In this problem set you work with cities infobox data, audit it, come up with a
cleaning idea and then clean it up.
在此习题集中，你将处理城市 infobox 数据，对数据进行审核，然后想出清理方法并清理数据。

In the previous quiz you recognized that the "name" value can be an array (or
list in Python terms). It would make it easier to process and query the data
later if all values for the name are in a Python list, instead of being
just a string separated with special characters, like now.
在上一道测验中，你意识到“name”值可以是数组（或用 Python 术语来说的话，是列表）。
如果名称的所有值是 Python 列表（而不是用特殊字符分隔的字符串，例如现在的状况），则稍后更容易处理和查询数据。

Finish the function fix_name(). It will recieve a string as an input, and it
will return a list of all the names. If there is only one name, the list will
have only one item in it; if the name is "NULL", the list should be empty.
The rest of the code is just an example on how this function can be used.
请完成函数fix_name()。它将获得字符串输入，并返回所有名称列表。如果只有一个名称，列表将只有一项。
如果名称是“NULL”，该列表应该为空。代码的其余部分只是可以用来展示如何使用该函数的示例。
"""
import codecs
import csv
import pprint

CITIES = 'cities.csv'


def fix_name(name):

    # YOUR CODE HERE
    if name == 'NULL':
        name = []
    elif name[0] == '{':
        name = name[1:-1].split('|')
    else:
        name = [name]
    return name


def process_file(filename):
    data = []
    with open(filename, "r") as f:
        reader = csv.DictReader(f)
        #skipping the extra metadata
        for i in range(3):
            l = reader.next()
        # processing file
        for line in reader:
            # calling your function to fix the area value
            if "name" in line:
                line["name"] = fix_name(line["name"])
            data.append(line)
    return data


def test():
    data = process_file(CITIES)

    print "Printing 20 results:"
    for n in range(20):
        pprint.pprint(data[n]["name"])

    assert data[14]["name"] == ['Negtemiut', 'Nightmute']
    assert data[9]["name"] == ['Pell City Alabama']
    assert data[3]["name"] == ['Kumhari']

if __name__ == "__main__":
    test()

Printing 20 results:
['Kud']
['Kuju']
['Kumbhraj']
['Kumhari']
['Kunigal']
['Kurgunta']
['Athens']
['Demopolis']
['Chelsea Alabama']
['Pell City Alabama']
['City of Northport']
['Sand Point']
['Unalaska Alaska']
['City of Menlo Park']
['Negtemiut', 'Nightmute']
['Fairbanks Alaska']
['Homer']
['Ketchikan Alaska']
['Nuniaq', 'Old Harbor']
['Rainier Washington']


## 交叉字段审查

In [200]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
In this problem set you work with cities infobox data, audit it, come up with a
cleaning idea and then clean it up.
在此习题集中，你将处理城市 infobox 数据，对数据进行审核，然后想出清理方法并清理数据。

If you look at the full city data, you will notice that there are couple of
values that seem to provide the same information in different formats: "point"
seems to be the combination of "wgs84_pos#lat" and "wgs84_pos#long". However,
we do not know if that is the case and should check if they are equivalent.
如果你查看完整的城市数据，会发现有几个值似乎提供的是同一信息，只是格式不同：
“point”似乎是“wgs84_pos#lat”和“wgs84_pos#long”的结合体。但是，我们不知道是否是这种情况，
你应该检查它们是否相等。

Finish the function check_loc(). It will recieve 3 strings: first, the combined
value of "point" followed by the separate "wgs84_pos#" values. You have to
extract the lat and long values from the "point" argument and compare them to
the "wgs84_pos# values, returning True or False.
完成函数check_loc()。它应该获得 3 个字符串：首先是“point”的值后面跟上单独的“wgs84_pos#”值。
你应该从“point”参数中提取lat和长值，并将它们与“wgs84_pos#”值对比，返回 True 或 False。

Note that you do not have to fix the values, only determine if they are
consistent. To fix them in this case you would need more information. Feel free
to discuss possible strategies for fixing this on the discussion forum.
注意，你不需要修正这些值，只需判断它们是否保持一致。要修正这些值，你需要更多信息。欢迎在论坛上讨论如何修正这些值。

The rest of the code is just an example on how this function can be used.
Changes to "process_file" function will not be taken into account for grading.
代码的其余部分只是可以用来展示如何使用该函数的示例。我们在打分时，不会考虑对“process_file”函数做出的更改。
"""
import csv
import pprint

CITIES = 'cities.csv'


def check_loc(point, lat, longi):
    # YOUR CODE HERE
    return point == lat + ' ' + longi
    # pass


def process_file(filename):
    data = []
    with open(filename, "r") as f:
        reader = csv.DictReader(f)
        #skipping the extra matadata
        for i in range(3):
            l = reader.next()
        # processing file
        for line in reader:
            # calling your function to check the location
            result = check_loc(line["point"], line["wgs84_pos#lat"], line["wgs84_pos#long"])
            if not result:
                print "{}: {} != {} {}".format(line["name"], line["point"], line["wgs84_pos#lat"], line["wgs84_pos#long"])
            data.append(line)

    return data


def test():
    assert check_loc("33.08 75.28", "33.08", "75.28") == True
    assert check_loc("44.57833333333333 -91.21833333333333", "44.5783", "-91.2183") == False

if __name__ == "__main__":
    test()